In [17]:
import os
import numpy as np
import json
from bs4 import BeautifulSoup


def word_filter(words):      
   words = words.replace("\n", "")
   words = words.replace("\r", "")
   words = words.replace("&nbsp;", "")
   words = words.replace("	", "")
   words = words.replace(" ", "")
   words = words.strip()
   #s.replace('&nbsp;',' ')
   #article = re.sub(r'	', '', article)
   #article = re.sub(r' ', '', article)
   return words

def remove_subtitle(name):      
   name = name.replace(".html", "")
   name = name.replace(".pdf", "")
   name = name.replace(".docx", "")
   return name

# 轉換全部HTML成為JSON格式

# Handle files in the directory
html_dir = "./docs/input/html/"
json_dir = "./docs/output/0_json/"
paths = os.listdir('./docs/input/html/')

count = 0  # 初始化處理檔案數量計數器

for files in paths:
    
    # full_path = os.path.join(html_dir, files)
    # if not os.path.isfile(full_path):
    #     print(f"Skipping {files}, not a file.")
    #     continue
    
    files_id = [] 
    titles_id = [] 
    intro_id = []
    author_id = []
    articles_id = []
    ref_id=[]

    try:
        with open(html_dir + files, encoding="utf-8") as f:            
            #print('\r\n'+ '檔案名稱:' + files)              
            files_id.append(files)

            soup = BeautifulSoup(f, "html.parser")                                    
            # 尋找以文件起點<div class="alohascontent" id="ctl00_divContent">   
            all_content = soup.find("div", class_ = "alohascontent")             
            # print (all_content)  # 列印網頁內容，包含html tag 
      
            # 文章題目                                
            try:
                title = all_content.find("p", class_ = "alohastitle").text   #找文章的題目
                #print('題目:' + files + word_filter(title))
                titles_id.append(word_filter(title)) 
            except:
                try:   
                    title = soup.find("div", class_ = "info-title").text   #找文章的題目
                    #print('The above code is a comment written in Chinese characters. It appears to be a placeholder for a question or topic.
                    #print('題目:' + files + word_filter(title))
                    titles_id.append(word_filter(title)) 
                except:
                    print('錯誤-檔案找不到題目:' + files)
                    titles_id.append(np.nan)     
            
            # 文章引言                              
            try:
                introduction = all_content.find("em").text # 找文章引言<em>               
                #print('引言:' + word_filter(introduction))
                intro_id.append(word_filter(introduction))  
            except:
                try:
                    introduction = all_content.find("p", class_ = {"subtitle1","introduction"}).text # 找文章引言<em>
                    #print('引言:' + word_filter(introduction))
                    intro_id.append(word_filter(introduction))    
                except:
                    try:
                        introduction = all_content.find("p", class_ = "text-right author").findPrevious('p').text # 找文章引言<em>
                        #print('引言:' + word_filter(introduction))
                        intro_id.append(word_filter(introduction))
                    except:
                        try:
                            introduction = all_content.find("div", class_ = "clearfix").findNext('p').text # 找文章引言<em>
                            #print('引言:' + word_filter(introduction))
                            intro_id.append(word_filter(introduction))
                        except:
                            try:
                                introduction = soup.find("div", class_ = "content").findNext("div" , class_ = "wrap").findNext('p').text   #找文章的題目                                    
                                #print('引言:' + word_filter(introduction))
                                all_out.append(word_filter(introduction))
                            except:
                                #print('錯誤-文章找不到引言:' + files)
                                intro_id.append(np.nan)               
           
            # 作者
            try:
                author = all_content.find("p", class_ = {"text-right author","credit"}).text # 找文章引言<em>               
                #print('作者:' + word_filter(author))
                author_id.append(word_filter(author))  
            except:
                try:
                    author = all_content.find("div", class_ = {"subfont_12R"}).text # 找文章引言<em>               
                    #print('作者:' + word_filter(author))
                    author_id.append(word_filter(author))  
                except:
                    try:
                        author = all_content.find('span', attrs={"style": {"color:#8e44ad;font-size:0.933em;","color:#800080;font-size:0.933em;"}}).text # 找文章引言<em>               
                        #print('作者:"' + word_filter(author))
                        author_id.append(word_filter(author))  
                    except:
                        try:
                            author = soup.find("div", class_="content").findNext("div" , class_ = "wrap").findNext("div", class_="text-right").text   #找文章的題目                                    
                            #print('作者:' + word_filter(author))
                            author_id.append(word_filter(author)) 
                        except:
                            print('錯誤-文章找不到作者:' + files)
                            author_id.append(np.nan)

            # 內文/參考內文:
                  # 內文:

            try:
                for articles in all_content.find('p', attrs={"class": "text-right author"}).find_all_next("div",class_={"col-md-12","col-md-6"}):               
                    for articles1 in articles.find_all("p"):
                        #print('內文"1":' + word_filter(articles1.text))
                        articles_id.append(word_filter(articles1.text))
                for articles in all_content.find_all_next("div",class_={"caption","panel-body"}):                
                   #print('內文參考"1":' + word_filter(articles.text))  
                   ref_id.append(word_filter(articles.text))         
            except:
                try:
                    for articles in all_content.find('span', attrs={"style": {"color:#8e44ad;font-size:0.933em;"}}):
                        if articles.find_all_next("p", attrs={"align":"left"}):
                            for articles1 in articles.find_all_next("p", attrs={"align":"left"}):  
                                #print('內文"2":' + word_filter(articles1.text))
                                articles_id.append(word_filter(articles1.text))   
                        if articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs): 
                            for articles1 in articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs):  
                                #print('內文"2.1":' + word_filter(articles1.text))
                                articles_id.append(word_filter(articles1.text))
                                 
                        if articles.find_all_next("td", colspan={"2","3"}):
                            for articles1 in articles.find_all_next("td", colspan={"2","3"}):  
                                #print('內文參考"2.1":' + word_filter(articles1.text))
                                ref_id.append(word_filter(articles1.text))   
                        elif articles.find_all_next("p", style="text-align: left;"):
                            for articles1 in articles.find_all_next("p", style="text-align: left;"):  
                                #print('內文參考"2.2":' + word_filter(articles1.text)) 
                                ref_id.append(word_filter(articles1.text))  
                        elif articles.find_all_next("div", style="text-align: left;padding-left:520px;"):
                            for articles1 in articles.find_all_next("div", style="text-align: left;padding-left:520px;"):   
                                #print('內文參考"2.3":' + word_filter(articles1.text))
                                ref_id.append(word_filter(articles1.text))    
                        elif articles.find_all_next("span", style="font-size:0.933em;"):
                            for articles1 in articles.find_all_next("span", style="font-size:0.933em;"):   
                                #print('內文參考"2.4":' + word_filter(articles1.text))  
                                ref_id.append(word_filter(articles1.text))  

                except:
                    try:
                        for articles in all_content.find('span', attrs={"style": {"color:#800080;font-size:0.933em;"}}):
                            if articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs): 
                                for articles1 in articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs):  
                                    #print('內文"3":' + word_filter(articles1.text))
                                    articles_id.append(word_filter(articles1.text))
                            #elif articles.find_all("p", attrs={"align":"left"}):                  
                            if articles.find_all_next("span", style="font-size:0.933em;"):
                                for articles1 in articles.find_all_next("span", style="font-size:0.933em;"):   
                                    #print('內文參考"3.1":' + word_filter(articles1.text))  
                                    ref_id.append(word_filter(articles1.text))                                     
                            elif articles.find_all_next("div", style="text-align: left;padding-left:520px;"):
                                for articles1 in articles.find_all_next("div", style="text-align: left;padding-left:520px;"):   
                                    #print('內文參考"3.2":' + word_filter(articles1.text)) 
                                    ref_id.append(word_filter(articles1.text))   
                    except:
                        try:
                            for articles in all_content.find('div', attrs={"class": "subfont_12R"}):
                                for articles1 in articles.find_all_next("p",attrs={"class":"subfont_13"}):          
                                    #print('內文"4":' + word_filter(articles1.text))
                                    articles_id.append(word_filter(articles1.text))
                            for articles in all_content.find('div', attrs={"class": "subfont_12R"}):
                                for articles1 in articles.find_all_next("div",attrs={"class":"subfont_12"}):     
                                    #print('內文參考"4":' + word_filter(articles1.text))
                                    ref_id.append(word_filter(articles1.text))                 
                        except:
                            try:
                                for articles in all_content.find("p", attrs={"class": "credit"}):                        
                                    for articles1 in articles.find_all_next("p"):                                          
                                        #print('內文"5":' + word_filter(articles1.text))
                                        articles_id.append(word_filter(articles1.text))
                                    for articles1 in articles.find_all_next("figcaption"):     
                                        #print('內文參考"5":' + word_filter(articles1.text))
                                        ref_id.append(word_filter(articles1.text))                   
                            except:
                                try:
                                    #print('內文"5":' + files)                                
                                    for articles in soup.find('div', attrs={"class": "content"}).find_all_next("p"):     
                                        #print('內文"6":' + word_filter(articles.text))
                                        articles_id.append(word_filter(articles.text))  
                                    for articles in soup.find('div', attrs={"class": "content"}).find_all_next("div",class_="caption"):                #         print(articles.text)
                                        #print('內文參考"6":' + word_filter(articles.text))
                                        ref_id.append(word_filter(articles.text))                              
                                except:
                                    #print('錯誤-文章找不到內文/參考:' + files)
                                    articles_id.append(np.nan) 
                                    ref_id.append(np.nan) 
  
        dict = {"filename": files_id, "titles": titles_id, "intro": intro_id, "author": author_id, "articles": list(filter(None, articles_id)), "reference": list(filter(None, ref_id))}
        with open(json_dir + remove_subtitle(files) + ".json", "w", encoding="utf-8") as f:
            count += 1  # 成功處理一個檔案後，計數 +1
            print(f"已儲存: {files_id}")
            f.write(json.dumps(dict, indent=4, ensure_ascii=False))        
            

    except Exception as e:
        print(f"❌ 處理失敗: {files}，原因: {e}")
    
print(f"\n📂 共處理完成 {count} 個檔案")



已儲存: ['228事件(20).html']
已儲存: ['「回首向來蕭瑟處，歸去，也無風雨也無晴」—民國38年國軍遷臺紀事(30).html']
已儲存: ['「快速」發展的年代：麥克阿瑟公路通車一甲子(203).html']
已儲存: ['「日暮鄉關何處是」─「留越國軍」的返台路(31).html']
已儲存: ['「賽德克．巴萊」重現的霧社事件(17).html']
已儲存: ['「醫者仁也‧仁者人也」─光復初期臺灣醫學教育(29).html']
已儲存: ['ㄋㄟㄋㄟ補給站：美援牛奶的供應(127).html']
已儲存: ['一紙命令，臺灣命運大不同─中國台灣省行政長官公署警備總司令部第一號令(1).html']
已儲存: ['不用手機也可哈拉一整天─45年度公用電話擴充計畫(2).html']
已儲存: ['世界人權日(18).html']
已儲存: ['世界人權日─台灣人權政府相關檔案線上展示介紹(6).html']
已儲存: ['世紀糖鐵穿鄉越鎮的五分車(148).html']
已儲存: ['中元普渡與法國軍墓在基隆(170).html']
已儲存: ['中華商場：見證大臺北繁華歲月(166).html']
已儲存: ['中華航空：從軍事化管理到以客為尊經營(152).html']
已儲存: ['九年國教：春風化雨五十載(134).html']
已儲存: ['亞東關係協會：臺日友好關係的樞紐(185).html']
已儲存: ['人民頭家—公民直選總統(23).html']
已儲存: ['任重道遠─民國36年台灣省鐵路圖(11).html']
已儲存: ['保育與觀光—從國立公園到國家公園(141).html']
已儲存: ['元氣補給：美軍大兵在臺灣(163).html']
已儲存: ['光輝十月．榮耀臺灣(28).html']
已儲存: ['八七水災與家園重建(146).html']
已儲存: ['共榮併進—農復會與臺灣農業的發展(116).html']
已儲存: ['兵役節淺談兵役(21).html']
已儲存: ['再續絃歌：交通大學在臺灣的耕耘路(135).html']
已儲存: ['力挽狂瀾：民國51年副霍亂疫情撲滅與對日貿易(156).html']
已儲存: ['北迴鐵路串聯臺灣東部交通的動脈(198).html']
已儲存: ['原

In [18]:
import os
import json

json_dir = "./docs/output/0_json/"  # JSON檔案所在資料夾

# 取得所有 JSON 檔案
files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for file in files:
    full_path = os.path.join(json_dir, file)
    
    with open(full_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
        print(f"\n===== 檔案名稱: {file} =====")
        print(json.dumps(data, indent=2, ensure_ascii=False))


===== 檔案名稱: 228事件(20).json =====
{
  "filename": [
    "228事件(20).html"
  ],
  "titles": [
    "228事件"
  ],
  "intro": [
    "228事件發生迄今已逾一甲子，民國34年8月，第二次世界大戰結束，日本戰敗，正式宣佈無條件投降，臺灣終於脫離殖民地統治，回歸祖國懷抱…全臺人民原本歡欣鼓舞迎接新時代的到來，卻沒有想到一齣悲劇正慢慢地上演…民國36年2月27日傍晚7點半，一個意外的衝突，成了228事件的導火線…"
  ],
  "author": [
    "◎資料蒐集：應用服務組"
  ],
  "articles": [
    "光復之初，民眾表現出對國民政府熱烈的歡迎與支持。但由於二次大戰末期，臺灣屢遭美軍轟炸，公共設施遭受相當破壞，戰後生產原料取得不易，技術人才難求，短期內恢復戰前生產水準，自非易事。然而當時中央政府派遣來台灣的官員，集行政、司法、立法、軍事大權於一身的行政長官公署制，非但未能有效復原，臺政卻每況愈下，迭受抨擊，加上溝通不夠與觀念的差異，以及所採取的不良經濟、統治方式與風評不佳的官威軍政，種下釀成日後228事件的遠因。",
    "228事件發生的直接因素卻是偶然的。民國36年2月27日上午11時左右，臺灣省菸酒專賣局(後改稱臺灣省菸酒公賣局)接到密報，謂淡水有走私火柴、捲煙情事，乃派葉德根、傅學通等6名查緝員，會同警察大隊所派4名警察前往查緝，但在淡水僅查獲私煙5箱。又據密報，走私貨已移至臺北市天馬茶房(太平町，今之延平北路)附近。當查緝員抵達現場時，私販早已逃散，僅查獲一位40歲婦女林江邁的私煙，遂將其公私煙及現金悉數沒收。由於林江邁苦苦哀求，圍觀民眾亦紛紛加入求情行列，林江邁情急，抱住查緝員不放，遭查緝員以槍管打破頭而鮮血直流(查緝處理情形詳檔案影像)。",
    "目睹此景的群眾轉趨悲憤，查緝員見狀欲速離開，群眾卻緊追不捨。查緝員為求脫身而開槍示警，不幸誤中當時在自宅樓下看熱鬧的市民陳文溪，查緝員輾轉躲至警察總局，激憤的民眾搗毀其緝私卡車，要求交出兇手槍斃。民眾因不滿緝私人員打傷、打死人民而包圍憲警單位，次日(即2月28日)請願民眾卻遭長官公署衛兵開槍造成更大死傷。原本是單純的治安事件，因處置失當，而使民眾平日積